# 基于路由的工作任务查询

## 全局设置

In [21]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.chunk_size = 128
Settings.chunk_overlap = 10

CPU times: user 267 µs, sys: 35 µs, total: 302 µs
Wall time: 305 µs


In [2]:
%%time

import nest_asyncio
nest_asyncio.apply()

CPU times: user 1.26 ms, sys: 166 µs, total: 1.43 ms
Wall time: 1.19 ms


## 加载数据

In [28]:
%%time

# 模拟一个当天的全体人员工作日志
items=[
"开发部韩宇轩，simple-rag 产品，实现新特性：针对文档的摘要索引。通过摘要索引，为用户生成的总结性能有大幅度提升。经测试评估，准确率达到了95%，以前是45%。",
"开发部陈逸飞，simple-rag 产品，修正bug：中文断句不完整。之前断句不完整，造成嵌入相似度准确率不高。目前的做法是更换splitter实现，并且对分块参数做了优化。",
"开发部郑千，simple-rag 产品，修正bug：聊天界面 markdown 格式显示错误。bug的现象是部分字体显示不全，更换实现库，解决了这个问题。",
"产品部林晓琪，simple-rag 产品，提交新特性需求：多数据源的融合检索查询。用户反映以及我们测试，在某些功能下，大文档的检索召回率不高，造成回答效果不好。希望通过增加新的检索方法，并融合现在的检索结果，提高检索召回率。",
"产品部林晓琪，江南皮革厂客服助手项目，和客户开会，沟通需求。确定了客服助手的基本功能，并约定了下一次沟通时间和沟通内容。",
"运维部李泽言，江南皮革厂客服平台，系统扩容，解决了负载过高造成服务卡的问题。服务卡的主要原因是算力不足，原来是2张a100 40G，目前扩到了4张。今天业务高峰时间负载不到60%，问题解决。",
"销售部韩雪薇，天津化工集团后勤服务平台，准备标书商务部分，完成公司资质方面的文案。资质文案主要包括： 相关软件著作权证书、银行资质证明、国安资质证书和验资报告等，已经整理成word文件。",
"法务部张峻宁，完成审核了与深圳做大事公司的商务合同。之前对方提出了多个版权方面的条款，经过沟通，以及有关方面的查询，修改了相关条款，消除了商业风险，和客户达成了共识。目前已经提交给办公室。",
]

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 4.77 µs


In [29]:
from llama_index.core.schema import TextNode

nodes=[ TextNode(text=item, id_=f"node_{index}") for index, item in enumerate(items)]
nodes[0]

TextNode(id_='node_0', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='开发部韩宇轩，simple-rag 产品，实现新特性：针对文档的摘要索引。通过摘要索引，为用户生成的总结性能有大幅度提升。经测试评估，准确率达到了95%，以前是45%。', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

## 摘要索引

In [30]:
%%time


from llama_index.core import SummaryIndex

summary_index = SummaryIndex(nodes)

CPU times: user 393 µs, sys: 50 µs, total: 443 µs
Wall time: 436 µs


In [43]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

summary_retriever=summary_index.as_retriever()

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

summary_query_engine = RetrieverQueryEngine(
    retriever=summary_retriever,
    response_synthesizer=response_synthesizer,
)

response = query_engine.query(
    "给出涉及的所有部门所有人的任务摘要输出，输出示例如下："
    """
    产品部:

    - 张三
        - xxx产品，提交新特性需求: xxx
        - xxx项目，和客户开会，xxxxx
        - ...
    - 李四
        - xxxx
        - ...
    - 王五
      - ...

    开发部:
    
    - 赵六
      - xxx产品，修正bug：xxxx
      - xxx项目，实现新特性：xxxx
      ...
    - 于七
      - xxx产品，修正bug：xxxx
      - xxx项目，实现新特性：xxxx
      ...
    ...

    xxx部:
    ...
    """
)

print(str(response))

产品部:

- 林晓琪
    - simple-rag 产品，提交新特性需求：多数据源的融合检索查询。关注提高大文档的检索召回率。
    - 江南皮革厂客服助手项目，和客户开会，沟通需求并确定了基本功能。

开发部:
  
- 韩宇轩
    - simple-rag 产品，实现新特性：针对文档的摘要索引。优化总结生成性能，准确率提升至95%。
- 陈逸飞
    - simple-rag 产品，修正bug：中文断句不完整问题，通过更换splitter实现并优化分块参数解决此问题。
- 郑千
    - simple-rag 产品，修正bug：聊天界面 markdown 格式显示错误，更换实现库解决了显示不全的问题。

运维部:

- 李泽言
    - 江南皮革厂客服平台，系统扩容以解决负载过高造成的服务卡问题。通过增加算力（4张a100 40G）成功降低业务高峰时的负载至不到60%。

销售部:

- 韩雪薇
    - 天津化工集团后勤服务平台，准备标书商务部分，完成公司资质方面的文案整理成word文件。

法务部:

- 张峻宁
    - 审核与深圳做大事公司的商务合同。修改了涉及版权的条款以消除商业风险，并达成共识后提交给办公室。
CPU times: user 9.91 ms, sys: 0 ns, total: 9.91 ms
Wall time: 6.37 s


In [ ]:
%%time

response = query_engine.query("林晓琪完成哪些任务")

## 向量索引

In [41]:
%%time

from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)

CPU times: user 36.7 ms, sys: 0 ns, total: 36.7 ms
Wall time: 1.76 s


In [45]:
%%time

vector_retriever=vector_index.as_retriever()

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

vector_query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=response_synthesizer,
)

response = query_engine.query("林晓琪完成哪些任务")
print(str(response))

林晓琪完成了以下任务：
1. 提交了关于simple-rag产品的多数据源融合检索查询的新特性需求，以提高大文档的检索召回率。
2. 和江南皮革厂客服助手项目的客户开会沟通需求，确定了客服助手的基本功能，并约定了下一次沟通时间和内容。
CPU times: user 5.87 ms, sys: 3.76 ms, total: 9.63 ms
Wall time: 1.48 s


In [47]:
%%time

response = query_engine.query("具体说下多数据源融合检索查询的新特性需求")
print(str(response))

产品部林晓琪在simple-rag产品的开发过程中提出了新特性需求：多数据源的融合检索查询。用户反馈及测试结果显示，在某些功能下，大文档的检索召回率不高，影响了回答效果。为了提升性能，希望通过增加新的检索方法并整合现有的检索结果，从而提高检索召回率。
CPU times: user 9.7 ms, sys: 0 ns, total: 9.7 ms
Wall time: 1.54 s


## 基于单选的路由查询引擎

In [65]:
%%time


summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine(top_k=5)

CPU times: user 192 µs, sys: 0 ns, total: 192 µs
Wall time: 195 µs


In [66]:
%%time

from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "对工作任务总结方面的问题很有用。"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "适用于回答工作任务相关的具体问题。"
    ),
)

CPU times: user 13 µs, sys: 2 µs, total: 15 µs
Wall time: 16.2 µs


In [67]:
%%time

from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
)

CPU times: user 172 µs, sys: 0 ns, total: 172 µs
Wall time: 175 µs


### 摘要相关的问题

In [70]:
%%time

response = query_engine.query(
    "给出涉及的所有部门所有人的任务摘要输出，输出示例如下："
    """
    产品部:

    - 张三
        - xxx产品，提交新特性需求: xxx
        - xxx项目，和客户开会，xxxxx
        - ...
    - 李四
        - xxxx
        - ...
    - 王五
      - ...

    开发部:
    
    - 赵六
      - xxx产品，修正bug：xxxx
      - xxx项目，实现新特性：xxxx
      ...
    - 于七
      - xxx产品，修正bug：xxxx
      - xxx项目，实现新特性：xxxx
      ...
    ...

    xxx部:
    ...
    """
)

print(str(response))

产品部:

- 林晓琪
  - simple-rag产品，提交新特性需求：多数据源的融合检索查询。提升大文档检索召回率以改善回答效果。
  - 江南皮革厂客服助手项目，与客户沟通需求并确定基本功能。

开发部:

- 韩宇轩
  - simple-rag产品，实现新特性：针对文档摘要索引，提升用户生成的总结性能至95%准确率。
  
- 陈逸飞
  - simple-rag产品，修正bug：中文断句不完整问题。改进分句方式并优化分块参数。

运维部:

- 李泽言
  - 江南皮革厂客服平台，解决系统负载过高的问题。通过扩容算力至4张a100 GPU以提高服务稳定性和响应能力。

销售部:

- 韩雪薇
  - 天津化工集团后勤服务平台，完成标书商务部分的文案准备工作，整理公司资质文件如软件著作权证书、银行资质证明等。

法务部:

- 张峻宁
  - 审核并修改与深圳做大事公司的商务合同中的版权条款。通过沟通和查询消除商业风险，并达成共识后提交办公室。
CPU times: user 14.6 ms, sys: 307 µs, total: 14.9 ms
Wall time: 6.45 s


In [99]:
%%time

response = query_engine.query(
    "这是工作任务总结方面的问题"
    "林晓琪做了哪些工作任务，用json格式输出，比如："
    """
    {
        "name": "张三",
        "dept:: "开发部",
        "tasks": [
            "xx项目，实现新特性: xxxx",
             "xx产品，提交新特性: xxx",
             ...
        ]
    }
    """
    
)
print(str(response))

{
    "name": "林晓琪",
    "dept": "产品部",
    "tasks": [
        "simple-rag 产品，提交新特性需求: 多数据源的融合检索查询",
        "江南皮革厂客服助手项目，和客户开会，沟通需求。确定了客服助手的基本功能，并约定了下一次沟通时间和内容"
    ]
}
CPU times: user 14.3 ms, sys: 0 ns, total: 14.3 ms
Wall time: 2.75 s


In [102]:
%%time

response = query_engine.query(
    "这是工作任务总结方面的问题"
    "列表简要回答，林晓琪做了哪些工作任务"    
)
print(str(response))

在这份任务总结中，林晓琪主要负责以下几项工作：

1. **simple-rag产品**：提交了关于多数据源融合检索查询的新特性需求。此需求是为了提升大文档检索召回率，通过增加新的检索方法并融合现有结果来优化搜索体验。

2. **江南皮革厂客服助手项目**：与客户开会沟通需求，明确了客服助手的基本功能，并安排了下一次的沟通时间与内容。

这些任务显示林晓琪在产品开发和客户关系管理方面扮演着重要角色。
CPU times: user 10.2 ms, sys: 3.35 ms, total: 13.5 ms
Wall time: 3.45 s


### 具体问题

In [103]:
%%time

response = query_engine.query("具体说下多数据源融合检索查询的新特性需求")
print(str(response))

产品部林晓琪针对simple-rag产品提交了新特性的需求——多数据源的融合检索查询。这一需求旨在解决在某些功能下大文档检索召回率不高的问题，从而提升整体回答效果。通过引入新的检索方法并整合现有的检索结果，期望能够显著提高检索的召回率，确保用户能获取到更加全面和准确的信息。
CPU times: user 14 ms, sys: 3.8 ms, total: 17.8 ms
Wall time: 3.65 s
